In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ammi-dataset/Sample_Submission.csv
/kaggle/input/ammi-dataset/train.csv
/kaggle/input/ammi-dataset/test.csv


# Some discription --> Now here i am just implimenting the AMMI kaggle challange against which i am given some lables
Such that i have to pridict the price of wine
# Models Used -->Cuml and cudf Based
doing the preprocessing using the pandas Framework here
# For the Model building pourposes I am using the Cuml and cudf Models

In [2]:
#importing the Files here
import pandas as pd
train = pd.read_csv('../input/ammi-dataset/train.csv')
test=pd.read_csv('../input/ammi-dataset/test.csv')

In [3]:
#some cudf and cuml based models which help us out in implimenting the above challange using various models
import cupy as cp
import cuml
from cuml.linear_model import LogisticRegression
from cuml.ensemble import RandomForestClassifier as cuRFC
from cuml.naive_bayes import MultinomialNB
from cuml.svm import SVC
from cuml.linear_model import LogisticRegression
from cuml.multiclass import MulticlassClassifier
from cuml.multiclass import OneVsRestClassifier
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [4]:
train.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,Portugal,This is a fine rich balanced wine. It has ripe...,Vila Santa Reserva,88.870874,20.0,Alentejano,NaN,NaN,NaN,NaN,NaN,PORTUGUESE RED,J. Portugal Ramos,32027
1,France,"A solid, chunky wine, with a structure that is...",NaN,88.041695,28.0,Bordeaux,Lalande de Pomerol,NaN,NaN,NaN,NaN,BORDEAUX-STYLE RED BLEND,Château Tour Grand Colombier,71079
2,France,"This is powerful and concentrated, with the hi...",NaN,94.085021,130.0,Bordeaux,Saint-Émilion,NaN,NaN,NaN,NaN,BORDEAUX-STYLE RED BLEND,Château Figeac,32440
3,US,"Rich, ripe and oaky, this Petite Sirah charms ...",Thompson Vineyard,89.869797,34.0,California,Santa Barbara County,Central Coast,NaN,NaN,Jaffurs 2010 Thompson Vineyard Petite Sirah (S...,PETITE SIRAH,Jaffurs,124405
4,US,This wine is a unique in the state blend and f...,McKinley Springs Vineyard,89.017651,24.0,Washington,Horse Heaven Hills,Columbia Valley,Sean P. Sullivan,@wawinereport,Syncline 2016 McKinley Springs Vineyard Rosé (...,ROSé,Syncline,33649


In [5]:
test.head()

,index,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,41855,US,"Sweet new oak stands out, giving this Cabernet...",3D,95.036469,NaN,California,St. Helena,Napa,NaN,NaN,Salvestrin 2007 3D Cabernet Sauvignon (St. Hel...,CABERNET SAUVIGNON,Salvestrin,0
1,10328,France,Charming raspberry aromas intertwine with scen...,Heluicum,90.966405,NaN,Rhône Valley,Collines Rhôdaniennes,NaN,NaN,NaN,NaN,SYRAH,Les Vins de Vienne,1
2,60094,US,"Ripe blackberry, leather and soy show on the n...",Estate,88.964358,NaN,California,Santa Ynez Valley,Central Coast,Matt Kettmann,@mattkettmann,Brave & Maiden 2013 Estate Cabernet Franc (San...,CABERNET FRANC,Brave & Maiden,2
3,48333,US,White flowers and wild anise give this a crisp...,NaN,89.960356,NaN,California,Sonoma Coast,Sonoma,Virginie Boone,@vboone,Les Voleurs 2013 Chardonnay (Sonoma Coast),CHARDONNAY,Les Voleurs,3
4,14498,US,Pinot Gris has been making inroads in Washingt...,NaN,88.075501,NaN,Washington,Columbia Valley (WA),Columbia Valley,Sean P. Sullivan,@wawinereport,Chateau Ste. Michelle 2015 Pinot Gris (Columbi...,PINOT GRIS,Chateau Ste. Michelle,4


In [6]:
import cupy as np
from cuml import LinearRegression
df=pd.DataFrame(data=train,columns=['index','country','description','designation','points','price','province','region_1','region_2','taster_name','taster_twitter_handle','title','variety','winery','id'])

In [7]:
#train data set is converted to the data-frame
df

,index,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,NaN,Portugal,This is a fine rich balanced wine. It has ripe...,Vila Santa Reserva,88.870874,20.0,Alentejano,NaN,NaN,NaN,NaN,NaN,PORTUGUESE RED,J. Portugal Ramos,32027
1,NaN,France,"A solid, chunky wine, with a structure that is...",NaN,88.041695,28.0,Bordeaux,Lalande de Pomerol,NaN,NaN,NaN,NaN,BORDEAUX-STYLE RED BLEND,Château Tour Grand Colombier,71079
2,NaN,France,"This is powerful and concentrated, with the hi...",NaN,94.085021,130.0,Bordeaux,Saint-Émilion,NaN,NaN,NaN,NaN,BORDEAUX-STYLE RED BLEND,Château Figeac,32440
3,NaN,US,"Rich, ripe and oaky, this Petite Sirah charms ...",Thompson Vineyard,89.869797,34.0,California,Santa Barbara County,Central Coast,NaN,NaN,Jaffurs 2010 Thompson Vineyard Petite Sirah (S...,PETITE SIRAH,Jaffurs,124405
4,NaN,US,This wine is a unique in the state blend and f...,McKinley Springs Vineyard,89.017651,24.0,Washington,Horse Heaven Hills,Columbia Valley,Sean P. Sullivan,@wawinereport,Syncline 2016 McKinley Springs Vineyard Rosé (...,ROSé,Syncline,33649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174995,NaN,US,"Named for the clone so beloved by winemakers, ...",Old Wente,93.965358,55.0,California,Russian River Valley,Sonoma,Virginie Boone,@vboone,Lynmar 2014 Old Wente Chardonnay (Russian Rive...,CHARDONNAY,Lynmar,51364
174996,NaN,Portugal,"This is a fresh, fruity and juicy wine. Shot t...",Bojador Branco,85.975164,14.0,Alentejano,NaN,NaN,Roger Voss,@vossroger,Espaço Rural 2014 Bojador Branco White (Alente...,PORTUGUESE WHITE,Espaço Rural,87223
174997,NaN,Spain,"Clean and toasty, with mineral and the slighte...",Cordon Negro Brut,85.030109,10.0,Catalonia,Cava,NaN,NaN,NaN,NaN,SPARKLING BLEND,Freixenet,142217
174998,NaN,US,"A lovely Pinot Noir, showing flamboyantly ripe...",Estate Grown,90.867770,65.0,California,Russian River Valley,Sonoma,NaN,NaN,NaN,PINOT NOIR,Donum,64741


**Here we have to apply the LabelEncoder For which i am using the sklearn Library**

In [8]:
from sklearn.preprocessing import LabelEncoder
for col in train.columns:
    enc=LabelEncoder()
    train[col]=enc.fit_transform(train[col])
train.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,32,98899,35911,101546,16,8,1278,18,19,15,77411,457,8782,27483
1,14,10809,37931,92661,24,38,572,18,19,15,77411,59,4019,61033
2,14,104319,37931,170939,126,38,966,18,19,15,77411,59,3322,27839
3,42,68273,33838,117438,30,53,997,1,19,15,37566,423,8842,106750
4,42,112607,21691,108828,20,454,527,3,16,11,66873,497,14717,28885


In [9]:
#just dividing the dataset into the x and the target column that is price
x=train.drop(columns=["price"])
y=train['price']
y=np.array(y)
y=y.reshape(-1,1)

In [10]:
#displaying the columns of our splitted x data-frame
x.columns


Index(['country', 'description', 'designation', 'points', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery', 'id'],
      dtype='object')

In [11]:
#now apply the splitting here to ensure that whole of your dataset gets divided into x_train,y_train,x_test,y_test
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(x,y,train_size=0.2,random_state=42)

# 1. **Let's use some Machine Learning Models**

* **Apply Logistic Regression**

In [12]:
log_reg = OneVsRestClassifier(LogisticRegression())
log_reg.fit(X_train.astype('float32'),Y_train.astype('float32'))

[W] [10:04:45.678693] L-BFGS: max iterations reached
[W] [10:04:45.679039] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [10:04:45.688388] L-BFGS line search failed (code 4); stopping at the last valid step
[W] [10:04:45.697407] L-BFGS line search failed (code 4); stopping at the last valid step
[W] [10:04:45.705723] L-BFGS line search failed (code 4); stopping at the last valid step
[W] [10:04:45.713227] L-BFGS line search failed (code 4); stopping at the last valid step
[W] [10:04:45.720870] L-BFGS line search failed (code 4); stopping at the last valid step
[W] [10:04:45.728973] L-BFGS line search failed (code 4); stopping at the last valid step
[W] [10:04:45.736830] L-BFGS line search failed (code 4); stopping at the last valid step
[W] [10:04:45.744913] L-BFGS line search failed (code 4); stopping at the last valid step
[W] [10:04:45.752786] L-BFGS

OneVsRestClassifier()

**below we just want to predict the y values **

In [13]:
 y_pred_logistic = log_reg.predict(X_test.astype('float32'))

In [14]:
 y_pred_logistic=y_pred_logistic.astype('int32')

In [15]:
y_pred_logistic

array([ 11,  21,  11, ...,  11,  21, 164], dtype=int32)

**cuml accuracy score we are calculating here**

In [16]:
accuracy_score = cuml.metrics.accuracy_score(Y_train, y_pred_logistic)

In [17]:
accuracy_score

0.008750000037252903

# 2. **Knn**

In [18]:
 from cuml.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,Y_train)

KNeighborsClassifier()

In [19]:
knn_predict=knn.predict(X_test)

In [20]:
knn_predict

array([ 8, 41,  6, ...,  6, 32, 22])

In [21]:
KNN_accuracy_score = cuml.metrics.accuracy_score(Y_train, knn_predict )

**printing the knn accuracy score**

In [22]:
KNN_accuracy_score

0.007735714316368103

# Implimenting Linear Regression

In [23]:
lr = LinearRegression(fit_intercept = True, normalize = False,
                      algorithm = "eig")

In [24]:
reg = lr.fit(X_train.astype('float32'),Y_train.astype('float32'))

In [25]:
# make predictions based on model
prediction = lr.predict(X_test)

In [26]:
prediction=prediction.astype('int64')

In [27]:
prediction

array([26, 40, 16, ...,  4, 45, 43])

In [28]:
Linear_reg_accuracy_score = cuml.metrics.accuracy_score(Y_train, prediction )

In [29]:
Linear_reg_accuracy_score

0.00736428564414382

# **SVD**

In [30]:
lrs=LinearRegression(fit_intercept = True, normalize = False,
                      algorithm = "svd")

In [31]:
# make predictions based on model
prediction = lr.predict(X_test)


prediction=prediction.astype('int64')

prediction



Linear_reg_accuracy_score = cuml.metrics.accuracy_score(Y_train, prediction )



Linear_reg_accuracy_score


0.007428571581840515

# **QR Linear Regression Model**

In [32]:
lrsq=LinearRegression(fit_intercept = True, normalize = False,
                      algorithm = "qr")

In [33]:
# make predictions based on model
prediction = lr.predict(X_test)
prediction=prediction.astype('int64')
prediction
Linear_reg_accuracy_score = cuml.metrics.accuracy_score(Y_train, prediction )
Linear_reg_accuracy_score

0.007707142736762762

# **SVR-QD Model**

In [34]:
lrssq=LinearRegression(fit_intercept = True, normalize = False,
                      algorithm = "svd-qr")

In [35]:
# make predictions based on model
prediction = lr.predict(X_test)
prediction=prediction.astype('int64')
prediction
Linear_reg_accuracy_score = cuml.metrics.accuracy_score(Y_train, prediction )
Linear_reg_accuracy_score

0.009985714219510555

# **SVD-Jacobi **

In [36]:
lrssj=LinearRegression(fit_intercept = True, normalize = False,
                      algorithm = "svd-jacobi")

In [37]:
# make predictions based on model
prediction = lr.predict(X_test)
prediction=prediction.astype('int64')
prediction
Linear_reg_accuracy_score_svd = cuml.metrics.accuracy_score(Y_train, prediction )
Linear_reg_accuracy_score_svd

0.007321428507566452

In [38]:
#Submitting the results on Kaggle
!kaggle competitions submit -c ammi-bootcamp -f Abhinav_101903421_eval_ques.csv -m "Message"


Traceback (most recent call last):
  File "/opt/conda/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/opt/conda/lib/python3.7/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/opt/conda/lib/python3.7/site-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
